# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [34]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy
from sklearn import preprocessing

In [35]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation = []
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),len(self.centers),self.fitness,self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [36]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset = dataset.drop(dataset.columns[0], axis=1)
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [37]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [38]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2


In [39]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [40]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [41]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [42]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [43]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


### Standardization
In this cell, we standard our data using this equation: <img src='http://3.bp.blogspot.com/_xqXlcaQiGRk/RpO4yR0oKtI/AAAAAAAAABM/7rUWCMwizus/s200/fig2.png'></img>

In [44]:
x_bcw = preprocessing.scale(x_bcw)
x_wine = preprocessing.scale(x_wine)
x_cmc = preprocessing.scale(x_cmc)
#x_glass = preprocessing.scale(x_glass)
x_iris = preprocessing.scale(x_iris)
x_vowel = preprocessing.scale(x_vowel)

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [45]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500
variations_count = 15
number_of_clusters = 6 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [46]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [47]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_glass,sampling_interval = sampling_intervals(x_glass)))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [48]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #214 nodes, #0 centers, fitness=-1
 clusters:
[[ 1.51763 12.61     3.59     1.31    73.29     0.58     8.5      0.
   0.     ]
 [ 1.52152 13.12     3.58     0.9     72.2      0.23     9.82     0.
   0.16   ]
 [ 1.51915 12.73     1.85     1.86    72.69     0.6     10.09     0.
   0.     ]
 [ 1.51755 12.71     3.42     1.2     73.2      0.59     8.64     0.
   0.     ]
 [ 1.51754 13.39     3.66     1.19    72.79     0.57     8.27     0.
   0.11   ]] 
 Centers:
[]
 sampling_interval:
(array([ 1.53393, 17.38   ,  4.49   ,  3.5    , 75.41   ,  6.21   ,
       16.19   ,  3.15   ,  0.51   ]), array([ 1.51115, 10.73   ,  0.     ,  0.29   , 69.81   ,  0.     ,
        5.43   ,  0.     ,  0.     ]))


## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [49]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[ 1.51623 14.14     0.       2.88    72.61     0.08     9.18     1.06
   0.     ]
 [ 1.5161  13.42     3.4      1.22    72.69     0.59     8.32     0.
   0.     ]
 [ 1.51818 13.72     0.       0.56    74.45     0.      10.99     0.
   0.     ]
 [ 1.52664 11.23     0.       0.77    73.21     0.      14.68     0.
   0.     ]
 [ 1.52119 12.97     0.33     1.51    73.39     0.13    11.27     0.
   0.28   ]
 [ 1.51829 13.24     3.9      1.41    72.33     0.55     8.31     0.
   0.1    ]]
[[1.51589e+00 1.28800e+01 3.43000e+00 1.40000e+00 7.32800e+01 6.90000e-01
  8.05000e+00 0.00000e+00 2.40000e-01]
 [1.52300e+00 1.33100e+01 3.58000e+00 8.20000e-01 7.19900e+01 1.20000e-01
  1.01700e+01 0.00000e+00 3.00000e-02]
 [1.51618e+00 1.30100e+01 3.50000e+00 1.48000e+00 7.28900e+01 6.00000e-01
  8.12000e+00 0.00000e+00 0.00000e+00]
 [1.51215e+00 1.29900e+01 3.47000e+00 1.12000e+00 7.29800e+01 6.20000e-01
  8.35000e+00 0.00000e+00 3.10000e-01]
 [1.51719e+00 1.47500e+01 0.00000e+00 2.00000e+00 7.30200e+

## K-means Algorithm 
this function fitting k-means model on dataset

In [50]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    kmeans = kmeans.fit(x)
    labels = kmeans.labels_
    return (labels,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [51]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([3, 3, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, 5, 2,
       5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 2, 2, 5, 5, 3, 2,
       5, 3, 3, 2, 2, 3, 2, 3, 3, 5, 5, 5, 5, 5, 3, 3, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 5, 3, 5, 5, 3, 5, 3, 5, 5, 5, 3, 5, 3, 3, 5, 3,
       5, 5, 3, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 2, 2, 4, 4, 4, 4, 1,
       4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 3, 3, 3, 5, 2, 2, 2, 4, 4,
       3, 3, 3, 3, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 3, 3, 3, 5, 3, 2, 3, 3,
       5, 5, 3, 2, 3, 3, 3, 3, 2, 0, 2, 4, 4, 4, 1, 4, 4, 0, 0, 4, 2, 4,
       2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [52]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #214 nodes, #6 centers, fitness=422.75478626330533
 clusters:
[[1.51593e+00 1.32500e+01 3.45000e+00 1.43000e+00 7.31700e+01 6.10000e-01
  7.86000e+00 0.00000e+00 0.00000e+00]
 [1.51735e+00 1.30200e+01 3.54000e+00 1.69000e+00 7.27300e+01 5.40000e-01
  8.44000e+00 0.00000e+00 7.00000e-02]
 [1.51508e+00 1.51500e+01 0.00000e+00 2.25000e+00 7.35000e+01 0.00000e+00
  8.34000e+00 6.30000e-01 0.00000e+00]
 [1.51841e+00 1.30200e+01 3.62000e+00 1.06000e+00 7.23400e+01 6.40000e-01
  9.13000e+00 0.00000e+00 1.50000e-01]
 [1.51775e+00 1.28500e+01 3.48000e+00 1.23000e+00 7.29700e+01 6.10000e-01
  8.56000e+00 9.00000e-02 2.20000e-01]] 
 Centers:
[[ 1.51608107e+00  1.45557143e+01  1.53571429e-01  2.19357143e+00
   7.31192857e+01  5.09285714e-01  8.46357143e+00  9.16428571e-01
   1.39285714e-02]
 [ 1.51720569e+00  1.31300813e+01  3.49422764e+00  1.40170732e+00
   7.28017886e+01  5.96991870e-01  8.31504065e+00  4.47967480e-02
   6.33333333e-02]
 [ 1.51776500e+00  1.32725000e+01  0.0000000

In [53]:
sum_of_squares(candidates_array[0].nodes,candidates_array[0].centers,candidates_array[0].labels)

422.7547862633054

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [54]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [55]:
def center_dist(data, center, labels, center_position, center_cluster):
    sqe = 0
    for l in range(0,len(labels)):
        if center_cluster == labels[l] :
            data_l = data[l][center_position]
            resid = data_l - center
            sqe += (resid**2)
    return sqe

In [56]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = []
    for i in range(0,len(candidates_array_example.centers)):
        Second_layer = []
        for j in range(0,len(candidates_array_example.centers[i])):
            Second_layer.append(center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j,i))
        fitness_of_each_centers.append(Second_layer)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))

[[9.45934679e-05 1.60774857e+01 6.14404286e+00 8.21964286e+00
  2.21415857e+01 7.18919857e+01 1.90280429e+01 1.33504429e+01
  2.46678571e-02]
 [2.28067816e-04 2.10570992e+01 7.90740163e+00 1.13263415e+01
  2.59042065e+01 5.30378699e+00 3.80946748e+01 8.71766992e+00
  1.15813333e+00]
 [6.62410000e-06 3.77707500e+00 0.00000000e+00 1.10730000e+00
  1.63467500e+00 4.86367500e+00 3.21327500e+00 0.00000000e+00
  0.00000000e+00]
 [6.15245429e-05 6.73474286e+00 1.38050658e-30 2.00388571e+00
  1.06771429e+01 6.04085714e-01 7.02357143e+00 8.50500000e+00
  1.29942857e-01]
 [2.06685231e-05 1.13744923e+01 8.12709231e+00 1.12252308e+00
  4.10992308e+00 5.42507692e-01 7.21790769e+00 5.31692308e-02
  2.92492308e-01]
 [1.77721959e-04 1.08077436e+01 1.57272974e+01 6.34252308e+00
  1.13241436e+01 1.73517436e+00 1.38715590e+01 3.19035897e+00
  2.93707692e-01]]


# test paper own answer
## result:
**paper has wrong answer and numbers**

In [57]:
def just_test(data,center):
    temp1 = np.subtract(center,data)
    temp1 = np.power(temp1, 2)
    sqe = np.sum(temp1)
    return sqe


center =[[1.52434 , 12.03344 , 0.01215 , 1.12869 , 71.98256 , 0.19252 , 14.34306 , 0.23039 , 0.15156],
         [1.51956 , 13.25068 , 0.45229 , 1.53305 , 73.01401 , 0.38472 , 11.15803 , 0.00433 , 0.06599],
         [1.51362 , 13.15690 , 0.65548 , 3.13123 , 70.50411 , 5.33024 , 6.73773  , 0.67322 , 0.01490],
         [1.52132 , 13.74692 , 3.51952 , 1.01524 , 71.89517 , 0.21094 , 9.44764  , 0.03588 , 0.04680],
         [1.51933 , 13.08412 , 3.52765 , 1.36555 , 72.85826 , 0.57913 , 8.36271  , 0.00837 , 0.06182],
         [1.51567 , 14.65825 , 0.06326 , 2.21016 , 73.25324 , 0.02744 , 8.68548  , 1.02698 , 0.00382]]

label = []
for i in x_glass:
    test = []
    for j in center:
        test.append(just_test(i,j))
    label.append(np.argmin(test))
    
print(label)    
print(sum_of_squares(x_glass,center,label))
print(candidates_array[0].fitness)

[3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 1, 1, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3, 5, 5, 1, 1, 5, 4, 4, 4, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
343.7814274034
422.75478626330533


## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [58]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen

###########################################################################################################
[[9.46048196e-05 1.60774857e+01 3.36320965e+02 8.22306330e+00
  3.64038997e+01 3.34540043e+02 1.11571790e+02 1.33504429e+01
  2.47281142e-02]
 [2.05832288e-02 2.10570992e+01 7.90740163e+00 1.13263415e+01
  1.29457753e+02 5.30378699e+00 5.89266273e+03 2.41522731e+01
  1.96348785e+00]
 [5.80975499e-05 4.30550233e+00 9.18258373e+00 1.10730000e+00
  4.22570351e+01 3.10851726e+01 3.21327500e+00 4.16464779e+00
  5.90656467e-04]
 [6.15245429e-05 4.08535767e+01 8.02454075e+01 2.21294296e+00
  1.08646286e+01 6.04085714e-01 1.67123312e+02 9.44744529e+00
  1.29942857e-01]
 [2.76586046e-04 1.13744923e+01 1.22141917e+01 1.12252308e+00
  8.28135082e+00 1.51733335e+00 7.21790769e+00 5.31692308e-02
  3.41934749e-01]
 [2.10705998e-04 1.08077436e+01 2.31301935e+01 8.11386812e+00
  4.53335029e+01 6.37350156e+00 1.38715590e+01 3.19035897e+00
  2.93707692e-01]]
[[9.45934679e-05 1.60774857e+01 6.14404

# Probability Equation

In [59]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return

### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [60]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

# Shrink sampling interval
**here we shrink sampeling interval using reduction factor ro creat neighborhood of each center**

In [61]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[1.5052605714285714, 1.5269015714285716], [11.396964285714287, 17.714464285714286], [-1.9791785714285708, 2.2863214285714295], [0.6688214285714287, 3.7183214285714286], [70.45928571428571, 75.7792857142857], [-2.4404642857142855, 3.459035714285714], [3.3525714285714283, 13.57457142857143], [-0.5798214285714285, 2.4126785714285712], [-0.22832142857142856, 0.2561785714285714]], [[1.5063851910569106, 1.5280261910569104], [9.971331300813008, 16.288831300813005], [1.3614776422764225, 5.626977642276422], [-0.12304268292682918, 2.9264573170731705], [70.14178861788618, 75.46178861788617], [-2.352758130081301, 3.5467418699186988], [3.204040650406503, 13.426040650406504], [-1.4514532520325203, 1.5410467479674794], [-0.17891666666666667, 0.3055833333333333]], [[1.5069445, 1.5285855000000002], [10.113750000000001, 16.43125], [-2.13275, 2.13275], [-0.4897499999999997, 2.55975], [71.5875, 76.9075], [-2.03225, 3.86725], [5.2165, 15.438500000000001], [-1.4962499999999999, 1.4962499999999999], [-0.24

# variations
**Choose t diffrent random center of each cluster's center in shrinked sampling interval neghborhood**

In [62]:
def variation(candidates_array):
    for candidate in candidates_array:
        variations = []
        for i in range(0,variations_count):
            Second_layer = []
            for j in candidate.sampling_interval:
                Third_layer = []
                for k in range(0,len(j)):
                    #print(k)
                    Third_layer.append(random.uniform(j[k][0],j[k][1]))
                Second_layer.append(Third_layer)
            variations.append(Second_layer)
        candidate.variation = variations    
variation(New_candidates_array)
print(New_candidates_array[3].variation[1])

[[1.516982127244662, 15.69056842817885, 0.5367816619608734, 0.782233794638898, 70.24063976214724, 3.0636930250318906, 5.015910803422227, -1.19364347733935, -0.05370504466052724], [1.505513785547688, 12.495609534429542, 0.6067408910463781, 4.030927720493845, 71.51633790683854, 2.7703194080597884, 7.757277461975906, 2.0988079009658103, -0.10660166614720834], [1.525690704461719, 16.360247935719446, 2.0708467490495335, 2.082737338023547, 72.29556740405863, -2.4289476770010756, 5.399062147962279, 0.6856117651197755, 0.11411610831240279], [1.5237310416604186, 15.039314013259677, 2.5215103353648205, 2.339344646442405, 71.89504841113592, 2.352117522536313, 6.3160729915052745, -0.8801731053554455, -0.055876443741311344], [1.5177572223242457, 13.458125716772138, -0.7996986428180157, 2.7558178646138574, 70.76513497522484, 3.0982701505621244, 14.788646757642372, -0.374810133096123, 0.0766544228858666], [1.520904277672892, 14.540632545360033, 1.8282342724637846, 2.251415654193006, 75.32991887942613